In [1]:
import pandas as pd
import glob
import datetime

In [2]:
datapath = './Data'
files = glob.glob(datapath+'/*chain*.json')
files.sort()

# Extract Info From JSON Data

In [ ]:
def extract_data(data_in):
    option_type = data_in.columns[-2:]
    data_out = pd.DataFrame(columns=['optiontype', 'date', 'underlying_price', 'strike', 'description', 'delta', 'openinterest', 'total_investment'])
    underlying_price = data_in['underlyingPrice'][0]
    for option in option_type:
        dates = data_in[option].keys()
        for date in dates:
            openinterest = []
            description = []
            delta = []
            strikes = data_in[option][date].keys()
            for strike in strikes:
                openinterest.append(data_in[option][date][strike][0]['openInterest'])
                description.append(data_in[option][date][strike][0]['description'])
                delta.append(data_in[option][date][strike][0]['delta'])
            temp = pd.DataFrame()
            temp['openinterest'] = openinterest
            temp['strike'] = strikes
            temp['date'] = date.split(':')[0]
            temp['optiontype'] = option[:-10]
            temp['description'] = description
            temp['delta'] = delta
            data_out = data_out.append(temp)
    data_out['underlying_price'] = underlying_price
    data_out['total_investment'] = data_out.apply(lambda x: float(x.delta) * float(x.openinterest) * float(x.underlying_price) * 100, axis=1)
    return data_out

# Total Investment By Chain

In [ ]:
datapath = './Data'
def get_files(datapath):
    files = glob.glob(datapath+'/*chain*.json')
    files.sort()
    return files

In [ ]:
def get_total_investment(files):
    result = pd.DataFrame()
    date = []
    investment = []
    underlying_price = []
    for file in files:
        date.append(file.split('_')[2])
        data_in = pd.read_json(file)
        data_out = extract_data(data_in)
        investment.append(data_out['total_investment'].sum())
        underlying_price.append(data_out['underlying_price'].max())
    result['date'] = date
    result['investment'] = investment
    result['underlying_price'] = underlying_price
    return result

# Get Historical Investment

In [ ]:
files = get_files(datapath)
historical_investment = get_total_investment(files)

In [ ]:
historical_investment

# Get Today's Top Data

In [ ]:
data_in = pd.read_json(files[-1])
data_out = extract_data(data_in)

In [ ]:
top_calls = data_out[data_out['optiontype']=='call'].sort_values('total_investment', ascending=False)
# top_calls = top_calls[top_calls['openinterest']>=1000].reset_index(drop=True)

In [ ]:
top_calls.head(10)

In [ ]:
top_puts = data_out[data_out['optiontype']=='put'].sort_values('total_investment', ascending=True)
# top_puts = top_puts[top_puts['openinterest'] >= 1000].reset_index(drop=True)

In [ ]:
top_puts.head(10)

In [ ]:
top_options = data_out.sort_values('total_investment', ascending=False).head(10).reset_index(drop=True)

In [ ]:
top_options

In [ ]:
data_in = pd.read_json(files[-2])
d2 = extract_data(data_in)

In [ ]:
# data_out['name'] = data_out.apply(lambda x: (x.optiontype+'_'+x.date+'_'+x.strike), axis=1)
# d2['name'] = d2.apply(lambda x: (x.optiontype+'_'+x.date+'_'+x.strike), axis=1)
data_out = data_out[['description', 'total_investment']]
d2 = d2[['description', 'total_investment']]
merged_df = pd.merge(data_out, d2, on='description', suffixes=("_before", "_after"))
merged_df['change'] = merged_df.apply(lambda x: x.total_investment_after-x.total_investment_before, axis=1)

In [ ]:
top_delta = merged_df.sort_values('change', ascending=False).head(10).reset_index(drop=True)

In [ ]:
top_delta

In [ ]:
bottom_delta = merged_df.sort_values('change', ascending=True).head(10).reset_index(drop=True)

In [ ]:
bottom_delta

# Email Analysis

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
config = pd.read_json('./Data/config.json', typ='series')
recipients = pd.read_json('./Data/mail_to.json', typ='series')
timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
botmail = config["botmail"]
botpw = config["botpw"]
subject = 'AMZN Report - ' + timestamp

In [ ]:
def create_msg(historical_investment, top_calls, top_puts, top_delta, bottom_delta):
    report = '<h>Historical Investment</h>'
    report += historical_investment.head(10).to_html()

    report += '<br><br>'
    report += '<h>Top Calls</h>'
    report += top_calls.head(10).to_html()

    report += '<br><br>'
    report += '<h>Top Puts</h>'
    report += top_puts.head(10).to_html()

    report += '<br><br>'
    report += '<h>Top Deltas</h>'
    report += top_delta.head(10).to_html()

    report += '<br><br>'
    report += '<h>Top Deltas</h>'
    report += bottom_delta.head(10).to_html()
    return report

In [ ]:
for recipient in recipients:
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = botmail
    msg['To'] = recipient

    report = create_msg(historical_investment, top_calls, top_puts, top_delta, bottom_delta)

    msg.attach(MIMEText(report, 'html'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login(botmail, botpw)

    server.sendmail(botmail, recipient, msg.as_string())
    server.quit()